# Dependencias (posiblemente, ejecutar con precaución 💀)

In [1]:
%pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


# Trackeo

In [2]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



###################################################
#      PARTE DE TRACKEO DE POR COLOR DE OPENCV    #
###################################################
# Define los límites del color en el espacio HSV
# PUNTO A (WIP)
greenLower = (40, 30, 30);
greenUpper = (85, 255, 255);
# PUNTO B
pinkLower = (140, 30, 200)
pinkUpper = (170, 200, 255)

# Carga el video
nombre_video = 'YoYo1.mp4'
cap = cv2.VideoCapture('Videos/' + nombre_video) # Local
if not cap.isOpened():
  cap = cv2.VideoCapture('/content/' + nombre_video) # Colab

# Constante para conversión
# Medidas tomadas de pixeles y metros
medida_metros = 0.10
medida_pixeles = 68 ### DEPENDE DEL VIDEO # Cantidad de pixeles en 0.1 metros
const_XY = medida_metros/medida_pixeles

# Variables para guardar las posiciones y tiempos
positionsA = []
positionsA_m = []
positionsB = []
positionsB_m = []
fps = cap.get(cv2.CAP_PROP_FPS)  # Obtiene los FPS del video
frame_duration = 1.0 / fps  # Duración de cada frame en segundos
frame_count = 0

def trackearPosiciones(hsv, colorLower, colorUpper, positions, positions_m):
    # Crea una máscara para el color
    mask = cv2.inRange(hsv, colorLower, colorUpper)
    # Realiza una serie de dilataciones y erosiones para eliminar cualquier pequeño punto en la máscara
    #mask = cv2.erode(mask, None, iterations=2) # Los puntos marcados en el YoYo son pequeños, la erosion los elimina
    mask = cv2.dilate(mask, None, iterations=3)
    # Encuentra los contornos en la máscara
    contours, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        # Encuentra el contorno más grande en la máscara
        c = max(contours, key=cv2.contourArea)
        # Encuentra el centro del contorno
        M = cv2.moments(c)
        if M["m00"] > 0:
            center_x = int(M["m10"] / M["m00"])
            center_y = int(M["m01"] / M["m00"])
            center = (center_x, center_y)
            # Calcula el tiempo actual basado en el número de frames procesados
            current_time = frame_count * frame_duration
            positions.append((current_time, center_x, cap.get(cv2.CAP_PROP_FRAME_HEIGHT) - center_y))
            positions_m.append((current_time, center_x * const_XY, (cap.get(cv2.CAP_PROP_FRAME_HEIGHT) - center_y) * const_XY) )
            # Dibuja el contorno y el centro en el frame original
            cv2.drawContours(frame, [c], -1, (0, 255, 0), 2)
            cv2.circle(frame, center, 5, (0, 0, 255), -1)

while True:
    # Captura frame por frame
    ret, frame = cap.read()

    if not ret:
        break

    # Convierte el frame al espacio de color HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Trackear puntos segun color
    trackearPosiciones(hsv, greenLower, greenUpper, positionsA, positionsA_m)
    trackearPosiciones(hsv, pinkLower, pinkUpper, positionsB, positionsB_m)

    # Muestra el frame con el tracking
    #cv2.imshow('Object Tracking', frame)   # imshow() necesita display, no funciona en Colab

    # Rompe el bucle si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # Incrementa el contador de frames
    frame_count += 1

# Libera la captura y cierra las ventanas
cap.release()
cv2.destroyAllWindows()



##########################################################################################
#                                CREACION DE DATAFRAME                                   #
##########################################################################################
def crear_dataframe(positions):
    # Crea un DataFrame con las posiciones en funcion del tiempo que capturo el codigo de opencv
    df = pd.DataFrame(positions, columns=['Time (sec)', 'X', 'Y'])

    # Calcula las diferencias entre posiciones y tiempos, lo agrega al dataframe
    #diff: es una funcion que calcula la diferencia entre los elementos consecutivos, por ejemplo de tiempo en este caso.
    #fillna(0): remplaza los elementos vacios por un 0 para que no tire error.
    df['Delta_Time'] = df['Time (sec)'].diff().fillna(0)
    df['Delta_X'] = df['X'].diff().fillna(0)
    df['Delta_Y'] = df['Y'].diff().fillna(0)

    # Calcula la velocidad en X y Y a partir de las diferencias en posicion y el cambio en el tiempo, lo agrega al dataframe
    df['Speed_X'] = df['Delta_X'] / df['Delta_Time']
    df['Speed_Y'] = df['Delta_Y'] / df['Delta_Time']
    # Reemplaza inf y -inf por 0 en las velocidades en x e y (por si se va al infinito por dividir por cero por ejemplo)
    df['Speed_X'] = df['Speed_X'].replace([np.inf, -np.inf], 0)
    df['Speed_Y'] = df['Speed_Y'].replace([np.inf, -np.inf], 0)
    # Calcula la velocidad total, lo agrega al dataframe
    df['Speed'] = np.sqrt(df['Speed_X']**2 + df['Speed_Y']**2)
    # Reemplaza inf y -inf por 0 en la velocidad total
    df['Speed'] = df['Speed'].replace([np.inf, -np.inf], 0)

    # Calcular la aceleración en X e Y
    df['Acceleration_X'] = df['Speed_X'].diff() / df['Delta_Time']
    df['Acceleration_Y'] = df['Speed_Y'].diff() / df['Delta_Time']
    # Reemplaza inf y -inf por 0 en las aceleraciones en x e y
    df['Acceleration_X'] = df['Acceleration_X'].replace([np.inf, -np.inf], 0)
    df['Acceleration_Y'] = df['Acceleration_Y'].replace([np.inf, -np.inf], 0)
    # Calcula la aceleración total, lo agrega al dataframe
    df['Acceleration'] = np.sqrt(df['Acceleration_X']**2 + df['Acceleration_Y']**2)
    # Reemplaza inf y -inf por 0 en la aceleración total
    df['Acceleration'] = df['Acceleration'].replace([np.inf, -np.inf], 0)

    # Separar aceleracion y velocidad para eliminar ruido
    df_speed = df[['Time (sec)', 'Speed_X', 'Speed_Y', 'Speed']][df['Speed'] != 0].copy()
    df_acceleration = df[['Time (sec)', 'Acceleration_X', 'Acceleration_Y', 'Acceleration']][df['Acceleration'] != 0].copy()

    return df, df_speed, df_acceleration

def crear_dataframe(dfA, dfB):
    # Combina los DataFrames
    dfCombined = pd.merge(dfA, dfB, on='Time (sec)')

    # Calcula las diferencias, velocidades y aceleraciones
    dfCombined['Delta_Time'] = dfCombined['Time (sec)'].diff().fillna(0)
    dfCombined['Delta_X_A'] = dfCombined['X_A'].diff().fillna(0)
    dfCombined['Delta_Y_A'] = dfCombined['Y_A'].diff().fillna(0)
    dfCombined['Delta_X_B'] = dfCombined['X_B'].diff().fillna(0)
    dfCombined['Delta_Y_B'] = dfCombined['Y_B'].diff().fillna(0)

    # Calcula la posición relativa
    dfCombined['Relative_X'] = dfCombined['X_B'] - dfCombined['X_A']
    dfCombined['Relative_Y'] = dfCombined['Y_B'] - dfCombined['Y_A']

    # Calcula las velocidades
    dfCombined['Velocidad_X_A'] = dfCombined['Delta_X_A'] / dfCombined['Delta_Time']
    dfCombined['Velocidad_Y_A'] = dfCombined['Delta_Y_A'] / dfCombined['Delta_Time']
    dfCombined['Velocidad_X_B'] = dfCombined['Delta_X_B'] / dfCombined['Delta_Time']
    dfCombined['Velocidad_Y_B'] = dfCombined['Delta_Y_B'] / dfCombined['Delta_Time']

    # Calcula las aceleraciones
    dfCombined['Aceleracion_X_A'] = dfCombined['Velocidad_X_A'].diff() / dfCombined['Delta_Time']
    dfCombined['Aceleracion_Y_A'] = dfCombined['Velocidad_Y_A'].diff() / dfCombined['Delta_Time']
    dfCombined['Aceleracion_X_B'] = dfCombined['Velocidad_X_B'].diff() / dfCombined['Delta_Time']
    dfCombined['Aceleracion_Y_B'] = dfCombined['Velocidad_Y_B'].diff() / dfCombined['Delta_Time']

    #Separar aceleración y velocidad para eliminar ruido
    dfCombined_speed = dfCombined[['Time (sec)', 'Velocidad_Relativa_X', 'Velocidad_Relativa_Y']]
    dfCombined_acceleration = dfCombined[['Time (sec)', 'Aceleracion_Relativa_X', 'Aceleracion_Relativa_Y']]

    return dfCombined, dfCombined_speed, dfCombined_acceleration

# Creamos los DataFrames para los puntos
dfA, dfA_speed, dfA_acceleration = crear_dataframe(positionsA_m)
dfB, dfB_speed, dfB_acceleration = crear_dataframe(positionsB_m)
dfCombined, dfCombined_speed, dfCombined_acceleration = crear_dataframe(dfA, dfB)

# Guardamos los DataFrames actualizados en archivos CSV (es para chequear despues si todo esta en orden nomas)
dfA.to_csv('DataFrameA.csv', index=False, float_format='%.6f')
dfB.to_csv('DataFrameB.csv', index=False, float_format='%.6f')
dfCombined.to_csv('DataFrameB.csv', index=False, float_format='%.6f')
print("Datos de posiciones y velocidades guardados en 'dataframe.csv'.")



##############################################################################
#                         PLOTEO                                             #
##############################################################################
# Elegir dataframe a graficar (posiblemente cambie mas adelante)
dfG = dfB
dfG_speed = dfB_speed
dfG_acceleration = dfB_acceleration

# Crear figura y ejes
# 4 filas, 2 columnas
fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6), (ax7, ax8)) = plt.subplots(4, 2, figsize=(15, 20))

# Grafica la trayectoria
ax1.plot(dfG['X'], dfG['Y'], marker='o', linestyle='-', color='b')
ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.set_title('Trayectoria del Objeto')

# Grafica la velocidad
ax2.plot(dfG_speed['Time (sec)'], dfG_speed['Speed'], marker='o', linestyle='-', color='r', label='Velocidad Total')
ax2.set_xlabel('Tiempo (segundos)')
ax2.set_ylabel('Velocidad (metros/segundo)')
ax2.set_title('Velocidad del Objeto')
ax2.legend()

# Grafica la posición X en función del tiempo
ax3.plot(dfG['Time (sec)'], dfG['X'], marker='o', linestyle='-', color='g')
ax3.set_xlabel('Tiempo (segundos)')
ax3.set_ylabel('Posición X')
ax3.set_title('Posición X / Tiempo')
# Grafica la posición Y en función del tiempo
ax4.plot(dfG['Time (sec)'], dfG['Y'], marker='o', linestyle='-', color='m')
ax4.set_xlabel('Tiempo (segundos)')
ax4.set_ylabel('Posición Y')
ax4.set_title('Posición Y / Tiempo')

# Gráfica la velocidad en X e Y en función del tiempo
# Velocidad en X
ax5.plot(dfG_speed['Time (sec)'], dfG_speed['Speed_X'], marker='o', linestyle='-', color='blue')
ax5.set_xlabel('Tiempo (segundos)')
ax5.set_ylabel('Velocidad X (metros/segundo)')
ax5.set_title('Velocidad X / Tiempo')
# Velocidad en Y
ax6.plot(dfG_speed['Time (sec)'], dfG_speed['Speed_Y'], marker='o', linestyle='-', color='red')
ax6.set_xlabel('Tiempo (segundos)')
ax6.set_ylabel('Velocidad Y (metros/segundo)')
ax6.set_title('Velocidad Y / Tiempo')

# Graficar la aceleración en X e Y en función del tiempo
# Aceleración en X
ax7.plot(dfG_acceleration['Time (sec)'], dfG_acceleration['Acceleration_X'], marker='o', linestyle='-', color='green')
ax7.set_xlabel('Tiempo (segundos)')
ax7.set_ylabel('Aceleración X (metros/segundo^2)')
ax7.set_title('Aceleración X / Tiempo')
# Aceleración en Y
ax8.plot(dfG_acceleration['Time (sec)'], dfG_acceleration['Acceleration_Y'], marker='o', linestyle='-', color='purple')
ax8.set_xlabel('Tiempo (segundos)')
ax8.set_ylabel('Aceleración Y (metros/segundo^2)')
ax8.set_title('Aceleración Y / Tiempo')

# Ajustar el diseño
plt.tight_layout()
# Muestra los graficos
plt.show()


TypeError: crear_dataframe() missing 1 required positional argument: 'dfB'